# Granularity
Standardise the data such that they have the same granularity.  

 * Weekly from Week #17 2012
 * Locale : Singapore

Import modules:-

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import re
from datetime import date

### Set directory paths:-

In [ ]:
raw_path = "../../data/1_raw/"
out_path = "../../data/2_interim/"
clean_path = "../../data/3_clean/"

if not os.path.exists(out_path):
    os.makedirs(out_path)
if not os.path.exists(clean_path):
    os.makedirs(clean_path)

### Compile weather data into Weekly, Singapore average format

In [ ]:
weather_files = glob.glob(raw_path+'weather/*.csv')
len(weather_files)

#### Create DataFrame from file:

In [ ]:
dfWeather = pd.read_csv(weather_files.pop(), encoding='latin_1')
for f in weather_files:
    df = pd.read_csv(f, encoding='latin_1')
    dfWeather = dfWeather.append(df)

In [ ]:
dfWeather.describe()

In [ ]:
dfWeather.head()

#### Remove unecessary columns:

In [ ]:
dfWeather.columns

In [ ]:
dfWeather = dfWeather.drop(['Station', 'Highest 30 Min Rainfall (mm)', 
                            'Highest 60 Min Rainfall (mm)',
                           'Highest 120 Min Rainfall (mm)'], axis=1)

#### Rename column headers into usable labels, i.e. standard ASCII UTF-8 characters.

In [ ]:
dfWeather.columns

In [ ]:
dfWeather.columns = ['Year', 'Month', 'Day', 'Daily Rainfall Total',
       'Mean Temperature', 'Maximum Temperature',
       'Minimum Temperature', 'Mean Wind Speed',
       'Max Wind Speed']

In [ ]:
dfWeather.columns

#### Deal with missing / invalid data

In [ ]:
dfWeather = dfWeather.fillna(np.nan)

In [ ]:
for col in dfWeather:
    dfWeather[col] = pd.to_numeric(dfWeather[col], errors='coerce')

In [ ]:
dfWeather.describe()

#### Deal with improbable data:

In [ ]:
dfWeather['Minimum Temperature'] = dfWeather['Minimum Temperature'].apply(lambda x: np.nan if x==0 else x)

In [ ]:
dfWeather.describe()

#### Get week number:

In [ ]:
def get_weeknum(row):
    isocal = date(int(row['Year']), int(row['Month']), int(row['Day'])).isocalendar()
    weeknum = str(isocal[0]) + '-W' + "%02d"%isocal[1] 
    return weeknum

In [ ]:
dfWeather['Y-Week'] = dfWeather.apply(lambda row: get_weeknum(row), axis=1)

In [ ]:
dfWeather.head()

#### Recalculate data by week number:

In [ ]:
group_dict = {'Minimum Temperature':['min'], 
              'Daily Rainfall Total':['mean'], 
              'Mean Temperature':['mean'], 
              'Mean Wind Speed':['mean'], 
              'Maximum Temperature':['max'], 
              'Max Wind Speed':['max']}

In [ ]:
groupedWeather = dfWeather.groupby(['Y-Week']).agg(group_dict)


In [ ]:
groupedWeather.head()

In [ ]:
groupedWeather.columns

#### Rename columns:

In [ ]:
groupedWeather.columns = [col[0] for col in groupedWeather.columns]

In [ ]:
groupedWeather.columns

In [ ]:
groupedWeather.head()

In [ ]:
groupedWeather.rename(index=str, columns={'Daily Rainfall Total':'Mean Daily Rainfall Total'})

#### Save to file:

In [ ]:
groupedWeather.to_csv(out_path+'weather.csv')

# Dengue and Malaria data

In [ ]:
dfDisease = pd.read_csv(raw_path+"weekly-infectious-disease-bulletin-cases.csv", index_col=0)

In [ ]:
dfDisease = dfDisease[(dfDisease['disease']=='Malaria') |
                      (dfDisease['disease']=='Dengue Fever') | 
                      (dfDisease['disease']=='Dengue Haemorrhagic Fever')]

In [ ]:
dfDisease.head()

### Pivoting

In [ ]:
dfDisease = dfDisease.pivot(columns = 'disease', values = 'no._of_cases')

In [ ]:
dfDisease['Dengue'] = dfDisease['Dengue Fever']+dfDisease['Dengue Haemorrhagic Fever']

In [ ]:
dfDisease = dfDisease.drop(['Dengue Fever', 'Dengue Haemorrhagic Fever'], axis=1)

#### Save to file

In [ ]:
dfDisease.to_csv(out_path+'disease.csv')

# Combine Data

In [ ]:
dfData = pd.concat([groupedWeather, dfDisease], axis=1)

In [ ]:
dfData.head()

In [ ]:
dfData[-5:]

#### Remove invalid rows with no data:

In [ ]:
dfData = dfData.dropna(subset = ['Malaria','Dengue'], how='all')

In [ ]:
dfData.describe()

In [ ]:
dfData.head()

### Reformat indexing

In [ ]:
YearWeek = [[int(x) for x in item.split('-W')] for item in dfData.index]


In [ ]:
dfYearWeek = pd.DataFrame.from_records(YearWeek, index = dfData.index, columns = ['Year', 'Week'])

In [ ]:
dfDataYW = pd.concat([dfData,dfYearWeek], axis=1)

In [ ]:
dfDataYW = dfDataYW.set_index(['Year','Week'])

In [ ]:
dfDataYW.head()

#### Save to file

In [ ]:
dfDataYW.to_csv(clean_path+'data.csv')